# Exercício sobre otimização hiperparamétrica para regressão de preços de residências usando redes neurais densas (DNNs)

Neste exercício, você irá encontrar o melhor conjunto de hiperparâmetros de um modelo para regressão.

Leia atentamente o conteúdo de todas as células e sigas as intruções abaixo.

Ao final, responda:

+ Qual o melhor conjunto de hiperparâmetros encontrado?
+ Esse resultado é melhor do que aquele que encontramos em sala de aula?

## Importe as bibliotecas

Execute a célula abaixo.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
# Importamos a classe StandardScaler.
from sklearn.preprocessing import StandardScaler

# Instalando o KerasTuner.
!pip install keras-tuner --upgrade

# Importanda a biblioteca KerasTuner.
import keras_tuner as kt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 1.7 MB/s eta 0:00:00


## Baixe a base de dados

Execute a célula abaixo.

In [ ]:
data = tf.keras.datasets.boston_housing

(x_train, y_train), (x_test, y_test) = data.load_data()

57026/57026 [==============================] - 0s 0us/step


### Pradronize os dados

Execute a célula abaixo.

In [ ]:
# Instanciamos um objeto da classe StandardScaler.
scaler = StandardScaler()

# Calcula-se os parâmetros de padronização usando o conjunto de treinamento.
scaler.fit(x_train)

# Usamos o método `transform` para padronizar os atributos de treinamento e teste.
x_train_std = scaler.transform(x_train)
x_test_std = scaler.transform(x_test)

## Encontre os melhores hiperparâmetros

### Defina a função de criação do modelo


Modifique a função abaixo de forma que a busca aleatória teste combinações aleatórias dos seguintes hiperparâmetros e valores:

1.   Número de camadas ocultas (`layers`): 1, 2 e 3.
2.   Número de neurônios nas camadas ocultas (`units`): 5, 10, 15, 20 e 25.
3.   Passo de aprendizagem do otmizador (`learning_rate`): 0.0003, 0.001 e 0.003.
4.   Otimizador (`optimizer`): 'sgd' e 'adam'.
5.   Função de ativação das camadas ocultas (`activation`): 'relu' e 'sigmoid'.

**Observação**

+ A busca deve demorar vários minutos, tenha paciência.


In [ ]:
def build_model(hp):

  # Cria o modelo sequêncial.
  model = tf.keras.Sequential()

  # Adicionando a camada de entrada. Ela não possui pesos, apenas especifica as dimensões da entrada.
  model.add(tf.keras.layers.Input(13))


  # ADICIONE O SEU CÓDIGO AQUI.



  # Compila o modelo.
  model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

  # Retorna o modelo.
  return model

### Instancie o objeto de busca aleatória

Execute as células abaixo.

In [ ]:
# Instanciando um objeto da classe RandomSearch.
tuner = kt.RandomSearch(
    build_model,
    objective='val_loss'
)

#### Faça a busca pelo melhor modelo.

Execute as células abaixo.

In [ ]:
tuner.search(
    x_train_std, y_train,
    epochs=500,
    validation_data=(x_test_std, y_test)
)

Trial 10 Complete [00h 00m 59s]
val_loss: 15.436296463012695

Best val_loss So Far: 12.995327949523926
Total elapsed time: 00h 11m 46s


Obtendo os melhores hiperparâmetros.

In [ ]:
best_hps  = tuner.get_best_hyperparameters(1)[0]

print('A pesquisa de hiperparâmetros foi concluída.')
print(f"""O número ideal de camadas ocultas é {best_hps.get('layers')}.""")
for i in range(best_hps.get('layers')):
  print(f"""O número ideal de neurônios na camada oculta # {i+1} é {best_hps.get('units_'+str(i))}.""")
print(f"""O valor ideal do passo de aprendizagem é o {best_hps.get('learning_rate')}.""")
print(f"""O otimizador ideal é {best_hps.get('optimizer')}.""")
print(f"""A função de ativação ideal é a {best_hps.get('activation')}.""")

A pesquisa de hiperparâmetros foi concluída.
O número ideal de camadas ocultas é 3.
O número ideal de neurônios na camada oculta # 1 é 20.
O número ideal de neurônios na camada oculta # 2 é 10.
O número ideal de neurônios na camada oculta # 3 é 15.
O valor ideal do passo de aprendizagem é o 0.003.
O otimizador ideal é adam.
A função de ativação ideal é a relu.


Imprimindo o resumo dos resultados da busca.

In [ ]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 07 summary
Hyperparameters:
layers: 3
activation: relu
units_0: 20
optimizer: adam
learning_rate: 0.003
units_1: 10
units_2: 15
Score: 12.995327949523926

Trial 04 summary
Hyperparameters:
layers: 3
activation: relu
units_0: 10
optimizer: sgd
learning_rate: 0.003
units_1: 15
units_2: 20
Score: 13.027603149414062

Trial 09 summary
Hyperparameters:
layers: 3
activation: sigmoid
units_0: 15
optimizer: adam
learning_rate: 0.003
units_1: 5
units_2: 5
Score: 15.436296463012695

Trial 05 summary
Hyperparameters:
layers: 3
activation: relu
units_0: 10
optimizer: adam
learning_rate: 0.001
units_1: 15
units_2: 15
Score: 15.630149841308594

Trial 06 summary
Hyperparameters:
layers: 1
activation: sigmoid
units_0: 25
optimizer: sgd
learning_rate: 0.003
units_1: 25
units_2: 15
Score: 17.479297637939453

Trial 03 summary
Hyperparameters:
layers: 1
activation: sigmoid
units_0: 25
opt

## Obtenha e construa o melhor modelo

Execute a célula abaixo.

In [ ]:
best_model = tuner.get_best_models(1)[0]

best_model.build(input_shape=(13,))

best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 20)                280       
                                                                 
 dense_1 (Dense)             (None, 10)                210       
                                                                 
 dense_2 (Dense)             (None, 15)                165       
                                                                 
 dense_3 (Dense)             (None, 1)                 16        
                                                                 
Total params: 671
Trainable params: 671
Non-trainable params: 0
_________________________________________________________________


## Avalie o melhor modelo nos conjuntos de treinamento e teste.

Execute as células abaixo.

In [ ]:
train_eval = best_model.evaluate(x_train_std, y_train)

print ("MSE no conjunto de treinamento: {:.5}".format(train_eval[0]))
print ("MAE no conjunto de treinamento: {:.5}".format(train_eval[1]))

13/13 [==============================] - 0s 2ms/step - loss: 2.6891 - mae: 1.2368
[2.689089298248291, 1.2368288040161133]
MSE no conjunto de treinamento: 2.6891
MAE no conjunto de treinamento: 1.2368


In [ ]:
test_eval = best_model.evaluate(x_test_std, y_test)

print ("MSE no conjunto de teste: {:.5}".format(test_eval[0]))
print ("MAE no conjunto de teste: {:.5}".format(test_eval[1]))

4/4 [==============================] - 0s 4ms/step - loss: 12.9953 - mae: 2.3362
MSE no conjunto de teste: 12.995
MAE no conjunto de teste: 2.3362
